#### We are gonna use the racing car environmet so the idea is trying to get a car to drive around a randomly generated race tack

### Action Space

###### If continuous there are 3 actions :

0: steering, -1 is full left, +1 is full right
1: gas
2: breaking

###### If discrete there are 5 actions:

0: do nothing
1: steer left
2: steer right
3: gas
4: brake

###  Observation Space

A top-down 96x96 RGB image of the car and race track.

###  1- Import Dependencies

In [ ]:
!pip install gym[box2d]

In [24]:
import gym
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

###  2- Test Environment

In [21]:
env_name = 'CarRacing-v2'
env = gym.make(env_name, render_mode="human")

In [ ]:
env.reset() 

In [ ]:
env.action_space 

In [ ]:
env.observation_space  # check that are 3 values from 0 t0 255

In [ ]:
env.render() # reder allow render the actual enviroment in which we are working on

###### test the environment

In [ ]:
episodes = 5
for episode in range(1, episodes+1):  # we're gonna check 5 times
    state = env.reset() # reset the env so we have a initial set of observations 
    done = False
    score = 0 
    
    while not done:
        env.render() # visualise the enviroment 
        action = env.action_space.sample() # take an action randomicaly
        n_state, reward, done, info,_ = env.step(action) # apply an action to the enviroment 
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close() # close down the render frame

###  3- Train Model

In [ ]:
# instanciate our environment
env = gym.make(env_name) 
env = DummyVecEnv([lambda: env])

In [25]:
log_path = os.path.join('Training','logs')
model = PPO('CnnPolicy',env, verbose=1,tensorboard_log=log_path)  # create the model

Using cpu device
Wrapping the env in a VecTransposeImage.


In [ ]:
model.learn(total_timesteps=10000) # let's train for 10000 steps 

###  4- Save Model

In [30]:
PPO_Path = os.path.join('Training','Saved Models','PPO_SelfDrive_model')
model.save(PPO_Path)

In [31]:
del model # delete the model to load and che

In [33]:
model=PPO.load(PPO_Path,env)

Wrapping the env in a VecTransposeImage.


###  5- Evaluate and Test

In [ ]:
env = gym.make(env_name, render_mode="human")
evaluate_policy(model, env, n_eval_episodes=10, render=True) # render is to visualize

In [37]:
env.close()

Test the self driving with a model trained 428000 steps

In [ ]:
PPO_Path_2 = os.path.join('Training','Saved Models','PPO_428k_Driving_model')
model=PPO.load(PPO_Path_2,env)

In [ ]:
env = gym.make(env_name, render_mode="human")
evaluate_policy(model, env, n_eval_episodes=10, render=True) # render is to visualize

In [ ]:
# Instead to evaluate with a encapsulate function as before let's create a evaluation where we use the trained agent
episodes = 5
for episode in range(1, episodes+1):  
    observations,_ = env.reset() 
    done = False
    score = 0 
    
    while not done:
        env.render() 
        action,_ = model.predict(observations) # here for the action we use the prediction of our trained model
        observations, reward, done, info,_ = env.step(action) 
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close() 